In [1]:
from smart_open import open
from ini_twitter import get_tweepy
from tweepy import Stream, StreamListener
import json

api = get_tweepy()

In [7]:
Stream

tweepy.streaming.Stream

In [2]:
def printd(o):
    if isinstance(o,dict):
        print(json.dumps(o,indent=4,ensure_ascii=False))
    else:
        print(json.dumps(o._json, indent=4,ensure_ascii=False))

# Mining Twitter data

Important things:
* extended tweet_mode
* count to return

3 Options:
* Use the Streaming API
* Search/Download Users
* Follow Users, Post Tweets etc.

In [3]:
result = api.search("@akk",count=100,result_type="recent",tweet_mode="extended")

In [ ]:
?api.search

In [4]:
printd(result[0])

{
    "created_at": "Wed Dec 11 08:49:19 +0000 2019",
    "id": 1204684574283980800,
    "id_str": "1204684574283980800",
    "full_text": "@hmger_hoerby @Hallaschka_HH @akk @JoachimStamp @SerapGueler @ArminLaschet das sagen nicht nur liberale muslime, sondern auch \"strenggläubige\". auch frauen mit hijab sehen das so.\n\nnur der politische islam sieht das anders.",
    "truncated": false,
    "display_text_range": [
        75,
        223
    ],
    "entities": {
        "hashtags": [],
        "symbols": [],
        "user_mentions": [
            {
                "screen_name": "hmger_hoerby",
                "name": "Hoerby 🇩🇪 🇪🇺",
                "id": 4259969115,
                "id_str": "4259969115",
                "indices": [
                    0,
                    13
                ]
            },
            {
                "screen_name": "Hallaschka_HH",
                "name": "Andreas Hallaschka",
                "id": 3063892271,
                "id_str": "306

In [5]:
len(result)


100

In [6]:
result = api.user_timeline("@akk",tweet_mode="extended",count=100)

In [7]:
printd(result[0])

{
    "created_at": "Sat Dec 07 15:02:31 +0000 2019",
    "id": 1203328943253794816,
    "id_str": "1203328943253794816",
    "full_text": "@lietzkow @EskenSaskia Coole Schuhe😎",
    "truncated": false,
    "display_text_range": [
        23,
        36
    ],
    "entities": {
        "hashtags": [],
        "symbols": [],
        "user_mentions": [
            {
                "screen_name": "lietzkow",
                "name": "Joerg Mueller-L.",
                "id": 43737749,
                "id_str": "43737749",
                "indices": [
                    0,
                    9
                ]
            },
            {
                "screen_name": "EskenSaskia",
                "name": "Saskia Esken",
                "id": 1423622834,
                "id_str": "1423622834",
                "indices": [
                    10,
                    22
                ]
            }
        ],
        "urls": []
    },
    "source": "<a href=\"http://twitter.com/downlo

# More complex things

In [8]:
retrieved = set()
full_return = open("full_tweets.ndjson.gz","w")
userdict = {}
contentdict = {}
replytoIDset = set()
irretrievable = set()

def process(ro):
    if ro.id in retrieved:
        return True
    full_return.write(json.dumps(ro._json) + "\n")
    retrieved.add(ro.id)
    # noinspection PyProtectedMember
    if "retweeted_status" in ro._json:
        ro = ro.retweeted_status
        retrieved.add(ro.id)

    #extended_entities
        
    userdict[ro.author.__dict__["_json"]["screen_name"]] = ro.author.__dict__["_json"]
    contentdict[ro.id] = {
        "time": ro.created_at,
        "favs": ro.favorite_count,
        "id": ro.id,
        "replyto_id": ro.in_reply_to_status_id,
        "lang": ro.lang,
        "place": ro.place,
        "retweets": ro.retweet_count,
        "truncated": ro.truncated,
        "text": ro.full_text,
        "username": ro.author.screen_name,
        "quoted_id": False
    }
    if "quoted_status_id" in ro._json:
        contentdict[ro.id]["quoted_id"] = ro.quoted_status_id
    if ro.in_reply_to_status_id:
        replytoIDset.add(ro.in_reply_to_status_id)

        
def process_stream(ro):
    if ro.id in retrieved:
        return True
    full_return.write(json.dumps(ro._json) + "\n")
    retrieved.add(ro.id)
    # noinspection PyProtectedMember
    if "retweeted_status" in ro._json:
        ro = ro.retweeted_status
        retrieved.add(ro.id)

    #extended_entities
        
    userdict[ro.author.__dict__["_json"]["screen_name"]] = ro.author.__dict__["_json"]
    contentdict[ro.id] = {
        "time": ro.created_at,
        "favs": ro.favorite_count,
        "id": ro.id,
        "replyto_id": ro.in_reply_to_status_id,
        "lang": ro.lang,
        "place": ro.place,
        "retweets": ro.retweet_count,
        "truncated": ro.truncated,
        "text": ro.text,
        "username": ro.author.screen_name,
        "quoted_id": False
    }
    if "extended_tweet" in ro._json:
        contentdict[ro.id]["text"]=ro._json["extended_tweet"]["full_text"]
    if "quoted_status_id" in ro._json:
        contentdict[ro.id]["quoted_id"] = ro.quoted_status_id
    if ro.in_reply_to_status_id:
        replytoIDset.add(ro.in_reply_to_status_id)

def fetch_replies():
    print(" ", len(replytoIDset))
    buckets = [[]]
    for i in replytoIDset - retrieved:
        if len(buckets[-1]) < 100:
            buckets[-1].append(i)
        else:
            buckets.append([])
    for i, bucket in enumerate(buckets):
        if len(bucket) > 90:
            [process(tweet) for tweet in api.statuses_lookup(bucket, tweet_mode="extended")]
    for tweet in retrieved:
        if tweet in replytoIDset:
            replytoIDset.remove(tweet)
    for bucket in buckets:
        for tweet in bucket:
            if tweet not in retrieved and tweet in replytoIDset:
                irretrievable.add(tweet)
                replytoIDset.remove(tweet)


In [9]:
[process(r) for r in result]
print(userdict)

{'akk': {'id': 47620601, 'id_str': '47620601', 'name': 'A. Kramp-Karrenbauer', 'screen_name': 'akk', 'location': '', 'description': 'Parteivorsitzende der @CDU Deutschlands, Bundesministerin der Verteidigung', 'url': 'https://t.co/tbP4FHRcIl', 'entities': {'url': {'urls': [{'url': 'https://t.co/tbP4FHRcIl', 'expanded_url': 'http://www.kramp-karrenbauer.de/impressum', 'display_url': 'kramp-karrenbauer.de/impressum', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 90398, 'friends_count': 1235, 'listed_count': 897, 'created_at': 'Tue Jun 16 13:16:59 +0000 2009', 'favourites_count': 2037, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': True, 'statuses_count': 8027, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'C0DEED', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 

In [3]:
tweet_list = []

class MeinStreamLauscher(StreamListener):
    def on_status(self, status):
        tweet_list.append(status._json)
        process_stream(status)
    def on_error(self, status):
        print(status)
    def on_exception(self, exception):
        print(exception)


In [10]:
lauschinstanz=MeinStreamLauscher()

stream=Stream(auth = api.auth, listener=lauschinstanz,tweet_mode="extended")
stream.sample(languages=None)

NameError: name 'MeinStreamLauscher' is not defined

In [13]:
stopwrds=["Climate","Greta","FFF","IPCC","#climatechange","climatechange","GreenDeal"]
stream=Stream(auth = api.auth, listener=lauschinstanz)
stream.filter(track=stopwrds)

NameError: name 'lauschinstanz' is not defined

In [ ]:
printd(tweet_list[0])

# Rate Limit Status

In [14]:
printd(api.rate_limit_status()["resources"])

{
    "lists": {
        "/lists/list": {
            "limit": 15,
            "remaining": 15,
            "reset": 1576055094
        },
        "/lists/memberships": {
            "limit": 75,
            "remaining": 75,
            "reset": 1576055094
        },
        "/lists/subscribers/show": {
            "limit": 15,
            "remaining": 15,
            "reset": 1576055094
        },
        "/lists/members": {
            "limit": 900,
            "remaining": 900,
            "reset": 1576055094
        },
        "/lists/subscriptions": {
            "limit": 15,
            "remaining": 15,
            "reset": 1576055094
        },
        "/lists/show": {
            "limit": 75,
            "remaining": 75,
            "reset": 1576055094
        },
        "/lists/ownerships": {
            "limit": 15,
            "remaining": 15,
            "reset": 1576055094
        },
        "/lists/subscribers": {
            "limit": 180,
            "remaining": 180,
  

In [15]:
fetch_replies()

  8
